In [ ]:
using Pkg
include("/cluster/home/danare/git/Clustering/TSClustering.jl")
cd("/cluster/home/danare/git/Clustering")
Pkg.activate(".")
using .TSClustering
using Distances
using Clustering
using DelimitedFiles
using DataFrames
using DynamicAxisWarping
using Dates
using Statistics
using CSV
using PlotlyJS

# Heat Maps for Warping Matrix

In [ ]:
# config
path = "/cluster/home/danare/git/GENeSYS_MOD.data/Output/output_excel/Timeseries.xlsx"
config = "/cluster/home/danare/git/Clustering/data/config5.yml"
k=6;
country = "DE";
config = TSClustering.read_yaml_file(file=config);

In [ ]:
data_org = TSClustering.read_data(path=path, config=config);
data_clustering_org = TSClustering.create_clustering_matrix(config=config, CountryData=data_org);
# apply normalization
data = TSClustering.normalize_data(CountryData=data_org, config=config);
data_clustering = TSClustering.create_clustering_matrix(config=config, CountryData=data);

In [ ]:
eucl = TSClustering.define_distance(w=0, data_clustering=data_clustering, fast_dtw=false);
dtw = TSClustering.define_distance(w=1, data_clustering=data_clustering, fast_dtw=false);

In [ ]:
TSClustering.plot_basic_heatmap(matrix=dtw)

In [ ]:
TSClustering.plot_basic_heatmap(matrix=eucl)

In [ ]:
p = make_subplots(rows=2, 
cols=2, 
subplot_titles= ["Wind Offshore" "Wind Onshore" "Load" "PV"]
)

for (i, j, k) in zip([], [1,1,2,2], [1,2,1,2], [true,false,false,false])
    z = cor(dtw)
    add_trace!(p,
    heatmap(z=z,
    colorscale="PdBu",
    showscale=k,
    y=names(data_org[df]),
    x=names(data_org[df])),
    row=i, col=j)
end


relayout!(p)
TSClustering.format_layout(p=p, max_value=0)
p

In [ ]:
#x1 = create_clustering_matrix(time_series1=data["LOAD"][:,"DE"], distance="Euclidean");
#x2 = create_clustering_matrix(time_series1=data["WIND_ONSHORE_AVG"][:,"DE"], distance="Euclidean");
#x3 = create_clustering_matrix(time_series1=data["PV_AVG"][:,"DE"], distance="Euclidean");
#x4 = create_clustering_matrix(time_series1=data["WIND_OFFSHORE_SHALLOW"][:,"DE"], distance="Euclidean");
y1 = TSClustering.define_distance(w=w,data_clustering=data_clustering, fast_dtw=false);
y2 = TSClustering.define_distance(w=w,data_clustering=data["WIND_ONSHORE_AVG"][:,"DE"], fast_dtw=false);
y3 = TSClustering.define_distance(w=w,data_clustering=data["PV_AVG"][:,"DE"], fast_dtw=false);
y4 = TSClustering.define_distance(w=w,data_clustering=data["WIND_OFFSHORE_SHALLOW"][:,"DE"], fast_dtw=false);

In [ ]:
dtw

In [ ]:
p = PlotlyJS.make_subplots(rows=2, 
cols=2, 
vertical_spacing=0.02,
subplot_titles = ["LOAD" "PV"; "WindON" "WIND OFF"]
)

for (w,i) in zip([x1,x2,x3,x4], [(1,1), (1,2), (2,1), (2,2)])
    add_trace!(p,
    PlotlyJS.heatmap(z=w,
    y=1:365,
    x=1:365),
    row=i[1], col=i[2])
end


relayout!(p)
p

In [ ]:
PlotlyJS.savefig(p, "eucl.html")

In [ ]:
p = PlotlyJS.make_subplots(rows=2, 
cols=2, 
vertical_spacing=0.02,
subplot_titles = ["LOAD" "PV"; "WindON" "WIND OFF"]
)

for (w,i) in zip([y1,y2,y3,y4], [(1,1), (1,2), (2,1), (2,2)])
    add_trace!(p,
    PlotlyJS.heatmap(z=w,
    y=1:365,
    x=1:365),
    row=i[1], col=i[2])
end


relayout!(p)
p

In [ ]:
PlotlyJS.savefig(p, "dtw.html")

In [ ]:
cost, a,b = dtw(data["LOAD"][1:24,"DE"],data["LOAD"][25:48,"DE"], Euclidean())
a

In [ ]:
b

In [ ]:
y1[2,1]

## Find Cost Delta

In [ ]:
for (i,j,n) in zip([x1,x2,x3,x4], [y1,y2,y3,y4], ["LOAD", "Windon", "PV", "Windoff"])
    println("$n the delta is $(sum(j)-sum(i))")
end